RNN의 문제점  
- 기울기 소실(지날수록 없어진다...)
- 번역에는 사용하기 힘들다.  
  각 나라별 어순정보를 파악하기 힘들다  
컨텍스트벡터(context vector) - Encoder은 입력문장의 모든 단어들을 순차적으로 입력받고 모든 단어를 압축한 하나의 벡터

디코더에는 시작과 끝에 특수한 문자를 넣는다.

어텐션 매커니즘 - 문맥을 더 잘 반영하는 벡터를 생성하는 메커니즘

* 각 나라별 어순 정보 파악 힘들다.


## Seq2seq 구현하기

### LSTM Encoder

In [ ]:
import tensorflow as tf

In [ ]:
class Encoder(tf.keras.Model):
    '''
    seq2seq의 encoder
    '''
  def __init__(self, vocab_size, embedding_dim, enc_units):
    super(Encoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(enc_units)
    # return_sequences 매개변수를 기본값 False로 전달

  def call(self, x): # __call__ 과 비슷한 듯

    print("입력 shape:",x.shape)
    
    x=self.embedding(x)
    print("Embedding Layer를 거친 shape ", x.shape)

    output = self.lstm(x)
    print("LSTM Layer의 Output Shape :", output.shape)

    return output

In [ ]:
vocab_size = 30000
emd_size = 256
lstm_size = 512
batch_size = 1
sample_seq_len = 3

print("Vocab Size : {0}".format(vocab_size))
print("Embedding Size : {0}".format(emd_size))
print("LSTM Size : {0}".format(lstm_size))
print("Batch Size : {0}".format(batch_size))
print("Sample Sequence Length {0}\n".format(sample_seq_len))

Vocab Size : 30000
Embedding Size : 256
LSTM Size : 512
Batch Size : 1
Sample Sequence Length 3



In [ ]:
encoder = Encoder(vocab_size, emb_size, lstm_size)
sample_input = tf.zeros((batch_size, sample_seq_len))

sample_output = encoder(sample_input)
# 컨텍스트 벡터로 사용한 인코더 LSTM의 최종 State값

입력 shape: (1, 3)
Embedding Layer를 거친 shape  (1, 3, 256)
LSTM Layer의 Output Shape : (1, 512)


![](https://aiffelstaticprd.blob.core.windows.net/media/images/GN-4-L-6.max-800x600.jpg)

### LSTM Decoder

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(dec_units, return_sequences=True)
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.softmax = tf.keras.layers.Softmax(axis= -1)

  def call(self, x, context_v):
    # 디코더의 입력 x와 인코더의 컨텍스트 벡터를 인자로 받습니다.
    print("입력 shape :", x.shape)

    x = self.embedding(x)
    print("Embedding Layer를 거친 Shape:", x.shape)

    context_v = tf.repeat(tf.expand_dims(context_v, axis=1), repeats=x.shape[1], axis =1)
    x = tf.concat([x, context_v], axis = -1)
    print("Context Vector가 더해진 Shape :", x.shape)

    x = self.lstm(x)
    print("LSTM Layer의 Output Shape :", x.shape)

    output = self.fc(x)
    print("Decoder 최종 Output Shape :", output.shape)

    return self.softmax(output)

In [ ]:
print("vocab Size : {0}".format(vocab_size))
print("Embedding Size : {0}".format(emb_size))
print("LSTM Size : {0}".format(lstm_size))
print("Batch Size : {0}".format(batch_size))
print("Sample Sequence Length : {0}\n".format(sample_seq_len))

vocab Size : 30000
Embedding Size : 256
LSTM Size : 512
Batch Size : 1
Sample Sequence Length : 3



In [ ]:
decoder = Decoder(vocab_size, emb_size, lstm_size)
sample_input = tf.zeros((batch_size, sample_seq_len))

dec_output = decoder(sample_input, sample_output)
# Decoder, call(x, context_v)를 호출

입력 shape : (1, 3)
Embedding Layer를 거친 Shape: (1, 3, 256)
Context Vector가 더해진 Shape : (1, 3, 768)
LSTM Layer의 Output Shape : (1, 3, 512)
Decoder 최종 Output Shape : (1, 3, 30000)


![](https://aiffelstaticprd.blob.core.windows.net/media/images/GN-4-L-7.max-800x600.jpg)

Rnn에 기반한 seq2seq 모델 두가지 문제점
1. 기억소실  
2. 하나의 고정된 벡터에 모든 정보를 압축하려다 보니 정보 손실이 발생

## 어텐션 메커니즘(Attention Mechanism)

- 어텐션의 아이디어는 디코더에서 출력 단어를 예측하는 매시점(time step)마다 인코더에서의 전체 입력 문장을 다시 한 번 참고한다는 점
- 전체 입력 문장을 전부 다 동일한 비율로 참고하는 것이 아니라, 해당 시점에서 예측해야할 단어와 연관이 있는 입력 단어부분을 좀 더 집주해서 보자.

In [ ]:
dict = {"2017" : "Transformer", "2018" : "BERT"}

In [ ]:
print(dict["2017"])

Transformer


In [ ]:
print(dict["2018"])

BERT


![](https://wikidocs.net/images/page/22893/%EC%BF%BC%EB%A6%AC.PNG)

Attention(Q, K, V) = Attention Value

- Query : t 시점의 디코더 셀에서의 은닉 상태
- Key : 모든 시점의 인코더 셀의 은닉 상태
- Value : 모든 시점의 인코더 셀의 은닉 상태

### 닷 프로덕트 어텐션 (Dot-Product Attention)

![](https://wikidocs.net/images/page/22893/dotproductattention1_final.PNG)

### 1. 어텐션 스코어 구하기

![](https://wikidocs.net/images/page/22893/dotproductattention2_final.PNG)

$$score(s_t, h_i) = S_t^T h_i $$

$$e^t = [s_t^T h_1, ..., s_t^T h_N]$$

### 2. 소프트맥스(softmax)함수를 통해 어텐션 분포를 구함

![](https://wikidocs.net/images/page/22893/dotproductattention3_final.PNG)

$$a^t = softmax(e^t)$$

### 3. 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값을 구함

![](https://wikidocs.net/images/page/22893/dotproductattention4_final.PNG)

$$a_t = \sum_{i=1}^{N}{a_i^th_i}$$ 

### 4. 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결한다.(Concatenate)

![](https://wikidocs.net/images/page/22893/dotproductattention5_final_final.PNG)

### 5. 출력층 연산의 입력이 되는 st를 계산

![](https://wikidocs.net/images/page/22893/st.PNG)

$$ \tilde{s_{t}}=tanh(W_c[a_t ; s_t] + b_c)$$

### 6. st를 출력층의 입력으로 사용

$$ \hat{y_t}=Softmax(W_y\tilde{s_t}+b_y)$$

- Bahdanau Attention
$$ Score_{alignment} = W * tanh(W_{decoder}* H_{decoder} +W_{encoder}* H_{encoder}) $$

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W_decoder = tf.keras.layers.Dense(units)
    self.W_encoder = tf.keras.layers.Dense(units)
    self.W_combine = tf.keras.layers.Dense(1)

  def call(self, H_encoder, H_decoder):
    print("[H_encoder] Shape :", H_encoder.shape)

    H_encoder = self.W_encoder(H_encoder)
    print("[W_encoder X H_encoder] Shape:", H_encoder.shape)

    print("\n[H_decoder] Shape: ", H_decoder.shape)
    H_decoder = tf.expand_dims(H_decoder, 1)
    H_decoder = self.W_decoder(H_decoder)

    print("[W_decoder X H_decoder] Shape:", H_decoder.shape)

    score = self.W_combine(tf.nn.tanh(H_decoder+H_encoder))
    print("[Score_alignment]Shape :", score.shape)

    attention_weights = tf.nn.softmax(score, axis = 1)
    print("\n최종 weight : \n", attention_weights.numpy())

    context_vector = attention_weights * H_decoder
    context_vector = tf.reduce_sum(context_vector, axis = 1)

    return context_vector, attention_weights

W_size = 100

print("Hidden State를 {0}차원으로 Mapping\n".format(W_size))

attention = BahdanauAttention(W_size)

enc_state = tf.random.uniform((1, 10, 512))
dec_state = tf.random.uniform((1, 512))

_ = attention(enc_state, dec_state)

Hidden State를 100차원으로 Mapping

[H_encoder] Shape : (1, 10, 512)
[W_encoder X H_encoder] Shape: (1, 10, 100)

[H_decoder] Shape:  (1, 512)
[W_decoder X H_decoder] Shape: (1, 1, 100)
[Score_alignment]Shape : (1, 10, 1)

최종 weight : 
 [[[0.07261766]
  [0.10911326]
  [0.06998613]
  [0.1168275 ]
  [0.08299869]
  [0.09790228]
  [0.10768744]
  [0.14077783]
  [0.10282373]
  [0.09926549]]]


![](https://aiffelstaticprd.blob.core.windows.net/media/original_images/GN-4-L-9.jpg)

### Loung Attention

$$ Score(H_{target},H_{source}) = H_{target}^T * W_{combine} * H_{source} $$

In [ ]:
class LuongAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(LuongAttention, self).__init__()
    self.W_combine = tf.keras.layers.Dense(units)
  
  def call(self, H_encoder, H_decoder):
    print("[H_encoder] shape: ", H_encoder.shape)

    WH = self.W_combine(H_encoder)
    print("[W_encoder X H_encoder] shape :", WH.shape)

    H_decoder = tf.expand_dims(H_decoder, 1)
    alignment = tf.matmul(WH, tf.transpose(H_decoder, [0, 2, 1]))
    print("[Score_alignment] Shape :", alignment.shape)

    attention_weights = tf.nn.softmax(alignment, axis = 1)
    print("\n최종 weight : \n", attention_weights.numpy())

    attention_weights = tf.squeeze(attention_weights, axis = -1)
    context_vector = tf.matmul(attention_weights, H_encoder)

    return context_vector, attention_weights

emb_dim = 512

attention = LuongAttention(emb_dim)

enc_state = tf.random.uniform((1, 10, emb_dim))
dec_state = tf.random.uniform((1, emb_dim))

_ = attention(enc_state, dec_state)

## 최종 weight의 크기는 단어연산의 크기와 동일하게 가능 여기선 "10"

[H_encoder] shape:  (1, 10, 512)
[W_encoder X H_encoder] shape : (1, 10, 512)
[Score_alignment] Shape : (1, 10, 1)

최종 weight : 
 [[[4.45325812e-03]
  [7.42507109e-05]
  [3.36647296e-04]
  [2.15690989e-06]
  [4.97238189e-01]
  [1.21852165e-04]
  [6.61878512e-05]
  [2.27045442e-04]
  [4.97472733e-01]
  [7.71726900e-06]]]


## 양방향 LSTM과 어텐션 메커니즘 (IMDB리뷰데이터)

### IMDB 리뷰 데이터 전처리하기

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = vocab_size)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [ ]:
# 정수 인코딩이 되어 있어 패딩바로 시작
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in x_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, x_train))/len(x_train)))

리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


In [ ]:
max_len = 500
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

### 바다나우 어텐션

$$score(query, key) = V^Ttanh(W_1 key + W_2 query)$$

In [ ]:
import tensorflow as tf

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): 
    #query size (batch_size, hidden size)
    # hidde_with time axis shape (batch size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention weights shape == (batch size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis =1 )

    # context_vector shape after sum == (batch size, hidden size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

### 양방향 LSTM + 어텐션 메커니즘

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

In [ ]:
sequence_input = Input(shape =(max_len, ), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)

In [ ]:
# dropout - 0.5 -> 노드와 노드 사이의 가중치를 50% 날려버림.. 오버피팃 방지
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(embedded_sequences)

In [ ]:
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

In [ ]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


In [ ]:
state_h = Concatenate()([forward_h, backward_h])# 은닉 상태
state_c = Concatenate()([forward_c, backward_c])# 셀 상태

In [ ]:
attention = BahdanauAttention(64) # 가중치의 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

In [ ]:
dense1 = Dense(20, activation='relu')(context_vector)# sigmoid 안씀 왜?? 히든이어서
dropout = Dropout(0.5)(dense1)
output = Dense(1,activation="sigmoid")(dropout)# 여기서 출력에서만 sigmoid 쓸지 고려
model = Model(inputs=sequence_input, outputs = output)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=5, batch_size=256, validation_data=(x_test, y_test), verbose = 1)

Epoch 1/5
98/98 [==============================] - 497s 5s/step - loss: 0.4590 - accuracy: 0.7807 - val_loss: 0.2818 - val_accuracy: 0.8836
Epoch 2/5
98/98 [==============================] - 477s 5s/step - loss: 0.2459 - accuracy: 0.9099 - val_loss: 0.2820 - val_accuracy: 0.8833
Epoch 3/5
98/98 [==============================] - 477s 5s/step - loss: 0.1953 - accuracy: 0.9352 - val_loss: 0.3199 - val_accuracy: 0.8803
Epoch 4/5
98/98 [==============================] - 474s 5s/step - loss: 0.1613 - accuracy: 0.9484 - val_loss: 0.3495 - val_accuracy: 0.8763
Epoch 5/5
98/98 [==============================] - 473s 5s/step - loss: 0.1339 - accuracy: 0.9574 - val_loss: 0.4018 - val_accuracy: 0.8702


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))

782/782 [==============================] - 338s 432ms/step - loss: 0.4018 - accuracy: 0.8702

 테스트 정확도: 0.8702


**---->> 여기서부터 갑자기 에러 발생 NLP_7일차 실습하기(2)로 이동**

## seq2seq with attention 스페인-영어 번역기

### 데이터 준비하기

In [83]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

In [84]:
path_to_zip = tf.keras.utils.get_file('spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip', extract=True)

FileExistsError: ignored

In [ ]:
path_to_file = os.path.dirname(path_to_zip)+ "/spa-eng/spa.txt"

NameError: ignored

In [ ]:
with open(path_to_file, "r") as f:
  raw = f.read().splitlines()

print("Data Size: ", len(raw))
print("Example:")

for sen in raw[0:100][::20]: print(">>", sen)

### 데이터 전처리 : 정제하기